In [18]:
import gymnasium as gym
import numpy as np
import torch
from gymnasium.wrappers import RescaleAction

In [7]:
env = gym.make("Pendulum-v1")

In [12]:
min_action, max_action = -1, 1
env = RescaleAction(env, min_action, max_action)

In [21]:
env.observation_space.sample().shape

(3,)

In [ ]:
def plot_rewards(env: gym.Env):
    obs_shape = env.observation_space.sample().shape
    action_shpae = env.action_space.sample().shape

    for i, low, high in enumerate(zip(
        env.observation_space.low, env.observation_space.high
    )):
        t = np.linspace(low, high, 100)
        obs = np.zeros_like(obs_shape)

